# Emparejamiento Encabezado-Boletín 

In [1]:
from pymongo import MongoClient, UpdateOne
import os
import dateutil.parser as parser
import re
from datetime import timedelta, datetime
from bson.objectid import ObjectId
from pprint import pprint
from tqdm import tqdm

Datos a utilizar para emparejar los archivos con documentos en MongoDB

In [2]:
UNA_HORA = timedelta(hours=1)
CANDIDATOS = ['AMLO', 'JAMK', 'RAC']
metadatos = { 
    candidato : {'ruta_abs' : '','fnames' : [],'fechas_iso': [] }
    for candidato in CANDIDATOS
}

In [3]:
for candidato in CANDIDATOS:
    ruta_rel = os.path.join('..', 'Boletines', candidato, 'txt', 'preprocesados')
    metadatos[candidato]['ruta_abs'] = os.path.abspath(ruta_rel)
    metadatos[candidato]['fnames'] = os.listdir(metadatos[candidato]['ruta_abs'])
    print(metadatos[candidato]['ruta_abs'])
    pprint(metadatos[candidato]['fnames'][:3])
    print()

C:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\Boletines\AMLO\txt\preprocesados
['Boletín-campaña-002-1_020418.txt',
 'Boletín-campaña-004_040418.txt',
 'Boletín-campaña-005_040418.txt']

C:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\Boletines\JAMK\txt\preprocesados
['010418 JAMK miles siguen inicio campaña.txt',
 '010418 JAMK Yuc candidato cerca jóvenes.txt',
 '010418 JAMK Yuc presenta 7 compr Mex.txt']

C:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\Boletines\RAC\txt\preprocesados
['COMUNICADO_01-RAC-30-03-2018.txt',
 'COMUNICADO_02-RAC-31-03-2018.txt',
 'COMUNICADO_03-RAC-31-03-2018.txt']



Los archivos de AMLO tienen un formato "Boletín-campaña-000-DDMMYY.txt"

In [4]:
candidato = 'AMLO'
metadatos[candidato]['fechas_iso'] = [
    parser.parse(f"2018-{s[-8:-6]}-{s[-10:-8]} 12:00:00-06:00")
    for s in metadatos[candidato]['fnames']
]
metadatos[candidato]['fechas_iso'][:5]

[datetime.datetime(2018, 4, 2, 12, 0, tzinfo=tzoffset(None, -21600)),
 datetime.datetime(2018, 4, 4, 12, 0, tzinfo=tzoffset(None, -21600)),
 datetime.datetime(2018, 4, 4, 12, 0, tzinfo=tzoffset(None, -21600)),
 datetime.datetime(2018, 4, 4, 12, 0, tzinfo=tzoffset(None, -21600)),
 datetime.datetime(2018, 4, 5, 12, 0, tzinfo=tzoffset(None, -21600))]

Los archivos de JAMK tienen un formato "DDMMYY JAMK <encabezado\>.txt"

In [5]:
candidato = 'JAMK'
metadatos[candidato]['fechas_iso'] = [
    parser.parse(f"2018-{s[2:4]}-{s[:2]} 12:00:00-06:00")
    for s in metadatos[candidato]['fnames']
]
metadatos[candidato]['fechas_iso'][:5]

[datetime.datetime(2018, 4, 1, 12, 0, tzinfo=tzoffset(None, -21600)),
 datetime.datetime(2018, 4, 1, 12, 0, tzinfo=tzoffset(None, -21600)),
 datetime.datetime(2018, 4, 1, 12, 0, tzinfo=tzoffset(None, -21600)),
 datetime.datetime(2018, 5, 1, 12, 0, tzinfo=tzoffset(None, -21600)),
 datetime.datetime(2018, 6, 1, 12, 0, tzinfo=tzoffset(None, -21600))]

Los archivos de RAC tienen un formato "COMUNICADO 00 - RAC - DD-MM-YYY.txt"

In [6]:
candidato = 'RAC'
metadatos[candidato]['fechas_iso'] = [
    parser.parse(f"2018-{s[-11:-9]}-{s[-14:-12]} 12:00:00-06:00")
    for s in metadatos[candidato]['fnames']
]
metadatos[candidato]['fechas_iso'][:5]

[datetime.datetime(2018, 3, 30, 12, 0, tzinfo=tzoffset(None, -21600)),
 datetime.datetime(2018, 3, 31, 12, 0, tzinfo=tzoffset(None, -21600)),
 datetime.datetime(2018, 3, 31, 12, 0, tzinfo=tzoffset(None, -21600)),
 datetime.datetime(2018, 4, 1, 12, 0, tzinfo=tzoffset(None, -21600)),
 datetime.datetime(2018, 4, 1, 12, 0, tzinfo=tzoffset(None, -21600))]

## Conexión a MongoDB
Conexión del cliente mongo a la base de datos

In [7]:
urifile = open("mongo_uri.txt", 'r', encoding='utf-8')
uri = urifile.read()
client = MongoClient(uri)
db = client.boletines_db
pprint(db.boletines.find_one())

{'_id': ObjectId('5ec3089dd380e649eb619ce9'),
 'boletin': 'Juntos haremos historia\n'
            'BOLETÍN CAMPAÑA-009\n'
            'PIDE LÓPEZ OBRADOR A TRUMP QUE NO MALTRATE A LOS MEXICANOS, '
            '“QUEREMOS RESPETO MUTUO”\n'
            '- NI LAS AMENAZAS, NI LOS MILITARES A LA FRONTERA EVITARAN QUE '
            'MÉXICO QUIERA TENER UNA RELACIÓN DE AMISTAD CON EU: AMLO\n'
            'MATAMOROS, TAMAULIPAS, 06 DE ABRIL DE 2018.- Andrés Manuel López '
            'Obrador manifestó que México y Estados Unidos “somos pueblos que '
            'estamos hermanados, los gobiernos no tienen por qué separar a los '
            'pueblos, nosotros le pedimos al presidente Donald Trump con todo '
            'respeto que no maltrate a los mexicanos, no queremos racismo y '
            'discriminación, queremos respeto mutuo”.\n'
            'Recalcó que nunca se le faltará el respeto al pueblo '
            'estadounidense, pero México y su pueblo no será piñata de ningún '
       

## Emparejamiento

Creo la función de emparejamiento. En ella, intentaré buscar coincidencias de encabezados (en la base de datos) dentro de los documentos (en texto plano). Para no hacer una búsqueda exhaustiva, uso el formato de fecha para reducir los posibles emparejamientos.

In [21]:
def emparejador(metadatos, candidato, db):
    coincidencias = []
    update_ops = []
    for fname, fecha in tqdm(zip(metadatos['fnames'], metadatos['fechas_iso'])):
        contenido = ""
        full_fname = os.path.join(metadatos['ruta_abs'], fname)
        with open(full_fname, 'r', encoding='utf-8') as f:
            contenido = f.read()
        filtro = {
            "candidato": candidato,
            "fecha": 
            { 
                "$gt": fecha-UNA_HORA*2, 
                "$lt": fecha+UNA_HORA*2
            } 
        }
        proy = { "encabezado":1 }
        cursor = db.boletines.find(filtro, proy)
        # print(f'Buscando en {fname}')  
        # print(contenido[85:170].lower())
        # print('...')
        matched = False
        for i, doc in enumerate(cursor):
            # print('  Prueba ', i)
            match = re.search(
                re.escape(doc['encabezado'][:-1].lower()), 
                contenido.lower(),
                re.IGNORECASE
            )
            # match = pattern.match(contenido.lower())
            # print('Probando con: ', res['encabezado'].lower())
            if match:
                # print(f"    RESULTADO encontrado")
                # print(f"    {match}")
                # print(f"    _id: {res['_id']}")
                coincidencias.append((fname, doc['_id']))
                update_ops.append(UpdateOne(
                    {'_id': doc['_id']},
                    {'$set':{
                        'boletin':contenido,
                        'file_name':fname
                    }}
                ))
                matched = True
        if not matched:
            tqdm.write(f'Sin coincidencias para {fname}')
            coincidencias.append((fname, ''))
        # print('------------------------------\n')
    res_updates = db.boletines.bulk_write(update_ops)
    print(res_updates.bulk_api_result)
    return coincidencias

In [22]:
coincidencias = {
    candidato : emparejador(metadatos[candidato], candidato, db)
    for candidato in CANDIDATOS
}

85it [00:07, 11.62it/s]
1it [00:00,  7.99it/s]

{'writeErrors': [], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 0, 'nMatched': 85, 'nModified': 85, 'nRemoved': 0, 'upserted': []}


147it [00:12, 11.52it/s]
2it [00:00, 12.08it/s]

{'writeErrors': [], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 0, 'nMatched': 147, 'nModified': 147, 'nRemoved': 0, 'upserted': []}


116it [00:10, 11.32it/s]


{'writeErrors': [], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 0, 'nMatched': 116, 'nModified': 116, 'nRemoved': 0, 'upserted': []}


In [13]:
for candidato in CANDIDATOS[:]:
    pprint(coincidencias[candidato][:5])
    print()

[('Boletín-campaña-002-1_020418.txt', ObjectId('5ec3089dd380e649eb619ce3')),
 ('Boletín-campaña-004_040418.txt', ObjectId('5ec3089dd380e649eb619ce4')),
 ('Boletín-campaña-005_040418.txt', ObjectId('5ec3089dd380e649eb619ce5')),
 ('Boletín-campaña-006_040418.txt', ObjectId('5ec3089dd380e649eb619ce6')),
 ('Boletín-campaña-007_050418.txt', ObjectId('5ec3089dd380e649eb619ce7'))]

[('010418 JAMK miles siguen inicio campaña.txt',
  ObjectId('5ec3089dd380e649eb619dab')),
 ('010418 JAMK Yuc candidato cerca jóvenes.txt',
  ObjectId('5ec3089dd380e649eb619dac')),
 ('010418 JAMK Yuc presenta 7 compr Mex.txt',
  ObjectId('5ec3089dd380e649eb619dad')),
 ('010518 JAMK Edomex unidad.txt', ObjectId('5ec3089dd380e649eb619de8')),
 ('010618 Eurviel Avila Edomex Ecatepec.txt',
  ObjectId('5ec3089dd380e649eb619e13'))]

[('COMUNICADO_01-RAC-30-03-2018.txt', ObjectId('5ec3089dd380e649eb619d36')),
 ('COMUNICADO_02-RAC-31-03-2018.txt', ObjectId('5ec3089dd380e649eb619d37')),
 ('COMUNICADO_03-RAC-31-03-2018.txt', O

In [20]:
import csv
import errno
for candidato in CANDIDATOS:
    ruta_rel_match = os.path.join('..', 'Boletines', candidato, 'metadata')
    ruta_abs_match = os.path.abspath(ruta_rel_match)
    try:
        os.makedirs(ruta_abs_match) # cre
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
    fname_id = os.path.join(ruta_abs_match, 'FName_ObjectId.csv')
    with open(fname_id, 'w', encoding='utf-8', newline='') as f:
        csv_out=csv.writer(f)
        csv_out.writerow(('fname','ObjectId'))
        for row in coincidencias[candidato]:
            csv_out.writerow(row)
        

In [13]:
client.close()